In [1]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama lamini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.5/689.5 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd

In [4]:
from dotenv import load_dotenv
import lamini
import os
load_dotenv("/content/drive/MyDrive/DATA298/.env")
lamini.api_key = os.environ["LAMINI_API_KEY"]


In [ ]:
# !curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


- Run Ollama serve in terminal

In [ ]:
# !ollama pull llama2

Error: could not connect to ollama app, is it running?


In [5]:
jerry_df = pd.read_csv("/content/scripts.csv")

jerry_df2 = jerry_df[jerry_df['Character'] == 'JERRY']

jerry_dia = jerry_df2['Dialogue']

In [6]:
jerry_dia.to_csv("jerry_dialogue.csv")

In [7]:
# Load, chunk and index the contents of the blog.
loader = CSVLoader("jerry_dialogue.csv")

docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
seinfeld_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [9]:
retriever = seinfeld_db.as_retriever(search_kwargs={'k': 10})
docs = retriever.invoke("How much soup will the guy at the shop give me?")


In [10]:
print(len(docs ))
print(type(docs))
print(type(docs[0]))

10
<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [11]:
print(docs)

[Document(page_content=': 27771\nDialogue: So he just gets soup. He wants to save the meal. So now I got to do it all over again.', metadata={'row': 8118, 'source': 'jerry_dialogue.csv'}), Document(page_content=': 34289\nDialogue: What about the soup?', metadata={'row': 9813, 'source': 'jerry_dialogue.csv'}), Document(page_content=': 27770\nDialogue: All right, fine. Get the soup!', metadata={'row': 8117, 'source': 'jerry_dialogue.csv'}), Document(page_content=': 35409\nDialogue: Is it a lot of cream soups?', metadata={'row': 10099, 'source': 'jerry_dialogue.csv'}), Document(page_content=': 34600\nDialogue: Elaine, let the man make his soup!', metadata={'row': 9880, 'source': 'jerry_dialogue.csv'}), Document(page_content=": 32090\nDialogue: He's the busboy, you think he cares about the soup?", metadata={'row': 9240, 'source': 'jerry_dialogue.csv'}), Document(page_content=": 34269\nDialogue: There's only one caveat -- the guy who runs the place is a little temperamental, especially abou

In [12]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("Dialogue:")[-1]
    output += doc
    output += "\n\n"
  return output

print(custom_format(docs)) # testing

 So he just gets soup. He wants to save the meal. So now I got to do it all over again.

 What about the soup?

 All right, fine. Get the soup!

 Is it a lot of cream soups?

 Elaine, let the man make his soup!

 He's the busboy, you think he cares about the soup?

 There's only one caveat -- the guy who runs the place is a little temperamental, especially about the ordering procedure. He's secretly referred to as the Soup Nazi.

 Because if he catches us, we'll never be able to get soup again.

 No. We gotta go to the soup place.

 Uh,..we'll go to 3rd Avenue. So, can you come with us for lunch to the soup place?




In [13]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.


In [14]:
import json
import re
from pprint import pprint
import random
import numpy as np

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [15]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating celebrities. In particular, you are
a world-leading expert at answering questions like Jerry from the hit
sitcom Seinfield.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Jerry would say. I have
included some relevant samples of what he actually said in the show.
<</INST>>
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
{response}
""".strip()

In [16]:
question = "What's your favorite planet?"
answer = "Mars"
generate_training_prompt(question,answer)

"### Instruction: <<SYS>>\nYou are an expert at impersonating celebrities. In particular, you are\na world-leading expert at answering questions like Jerry from the hit\nsitcom Seinfield.\n<</SYS>>\n\n<<INST>>\nGenerate a response that sounds as close to what Jerry would say. I have\nincluded some relevant samples of what he actually said in the show.\n<</INST>>\n\n### Input:\nWhat's your favorite planet?\n\n### Context:\n Your pants!\n\n Hey.\n\n Wow. What are we celebrating?\n\n What about me?\n\n My car!\n\n My pants!\n\n Hey, what is going on over there?\n\n Really?\n\n Well, what about my car?\n\n Me? Why me?\n\n\n\n### Response:\nMars"

In [17]:
seinfeld_file = pd.read_csv("/content/scripts.csv")


In [18]:
seinfeld_file

,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season
0,0,JERRY,Do you know what this is all about? Do you kno...,1.0,S01E01,1.0
1,1,JERRY,"(pointing at Georges shirt) See, to me, that b...",1.0,S01E01,1.0
2,2,GEORGE,Are you through?,1.0,S01E01,1.0
3,3,JERRY,"You do of course try on, when you buy?",1.0,S01E01,1.0
4,4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",1.0,S01E01,1.0
...,...,...,...,...,...,...
54611,54611,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,S09E23,9.0
54612,54612,PRISONER 3,You suck - I'm gonna cut you.,23.0,S09E23,9.0
54613,54613,JERRY,"Hey, I don't come down to where you work, and ...",23.0,S09E23,9.0
54614,54614,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,S09E23,9.0


In [19]:
seinfeld_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54616 entries, 0 to 54615
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  54616 non-null  int64  
 1   Character   54616 non-null  object 
 2   Dialogue    54605 non-null  object 
 3   EpisodeNo   54616 non-null  float64
 4   SEID        54616 non-null  object 
 5   Season      54616 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
# questions = []
# answers = []
# character_list = seinfeld_file['Character'].tolist()
# dialogue_list = seinfeld_file['Dialogue'].tolist()
# for i, character in enumerate(character_list[1:]):
#   if character == 'JERRY':
#     if character_list[i] != 'JERRY':
#       q = dialogue_list[i]
#       if "?" in q: # search if ? in the question
#         questions.append(dialogue_list[i]) #not jerry
#         answers.append(dialogue_list[i+1]) # jerry

In [20]:
questions = []
answers = []
character_list = seinfeld_file['Character'].tolist()
dialogue_list = seinfeld_file['Dialogue'].tolist()
for i, character in enumerate(character_list[1:]):
  if character == 'JERRY':
    if character_list[i] != 'JERRY':
      questions.append(dialogue_list[i])
      answers.append(dialogue_list[i+1])



In [21]:
len(questions)

14235

In [22]:
len(answers)

14235

In [23]:
set([type(x) for x in questions])

{float, str}

In [24]:
[x for x in questions if type(x) == type(1.0)]

[nan, nan]

In [25]:
for j in range(10):
  question = questions[j]
  answer = answers[j]
  generate_training_prompt(question,answer)

In [ ]:
# #Get fixed and the same test_data each time
# p=.8

# tmpList = []
# trainList = []
# testList = []
# for j,(question,answer) in enumerate(zip(questions,answers)):
#   if type(question) != str:
#     continue
#   tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)}
#   tmpList.append(tmp_dict)
#   if j == 200:
#     break
#   if j % 100 == 0:
#     print(j)
#   if j >p*len(questions) :
#     testList.append(tmp_dict)
#   else:
#     trainList.append(tmp_dict)

# data = Dataset.from_list(tmpList)
# train_data = Dataset.from_list(trainList)
# test_data = Dataset.from_list(testList)


In [26]:
tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questions,answers)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)}
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 100 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200


In [27]:
data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)

In [28]:
tmpList[:5]

[{'question': 'Are you through?',
  'answer': 'You do of course try on, when you buy?',
  'training_prompt': "### Instruction: <<SYS>>\nYou are an expert at impersonating celebrities. In particular, you are\na world-leading expert at answering questions like Jerry from the hit\nsitcom Seinfield.\n<</SYS>>\n\n<<INST>>\nGenerate a response that sounds as close to what Jerry would say. I have\nincluded some relevant samples of what he actually said in the show.\n<</INST>>\n\n### Input:\nAre you through?\n\n### Context:\n Are you through?\n\n (In pain) Oh, are you about done?\n\n Are we finished?\n\n So it's all over?\n\n So it's over?\n\n It's over?\n\n Really? Its over?\n\n What's over?\n\n Oh, yeah? Well, then, we're through! And you're fired!\n\n Yeah. are you full?\n\n\n\n### Response:\nYou do of course try on, when you buy?"},
 {'question': 'Yes, it was purple, I liked it, I dont actually recall considering the buttons.',
  'answer': 'Oh, you dont recall?',
  'training_prompt': '### 

In [29]:
notebook_login()

In [30]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [31]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [32]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [33]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [34]:
OUTPUT_DIR = "experiments"

# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

In [35]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [36]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/11372 [00:00<?, ? examples/s]

Map:   0%|          | 0/2861 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [37]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
284,0.756300,0.995785
568,0.878000,0.778803
852,0.393100,0.658595
1136,0.576100,0.590412
1420,0.403400,0.578974


TrainOutput(global_step=1420, training_loss=0.7320051585284757, metrics={'train_runtime': 5923.0352, 'train_samples_per_second': 3.84, 'train_steps_per_second': 0.24, 'total_flos': 1.3794522930266112e+17, 'train_loss': 0.7320051585284757, 'epoch': 2.0})

In [38]:
trainer.save_model()

In [39]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [40]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = trained_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [ ]:
# from google.colab import files
# files.download("/content/merged_model/model-00001-of-00003.safetensors")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import files
# files.download("/content/merged_model/model-00002-of-00003.safetensors")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import files
# files.download("/content/merged_model/model-00003-of-00003.safetensors")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !zip -r /content/experiments.zip /content/experiments

  adding: content/experiments/ (stored 0%)
  adding: content/experiments/special_tokens_map.json (deflated 75%)
  adding: content/experiments/checkpoint-711/ (stored 0%)
  adding: content/experiments/checkpoint-711/special_tokens_map.json (deflated 75%)
  adding: content/experiments/checkpoint-711/optimizer.pt (deflated 9%)
  adding: content/experiments/checkpoint-711/tokenizer.model (deflated 55%)
  adding: content/experiments/checkpoint-711/training_args.bin (deflated 48%)
  adding: content/experiments/checkpoint-711/adapter_model.bin (deflated 7%)
  adding: content/experiments/checkpoint-711/adapter_config.json (deflated 48%)
  adding: content/experiments/checkpoint-711/adapter_model.safetensors (deflated 7%)
  adding: content/experiments/checkpoint-711/README.md (deflated 67%)
  adding: content/experiments/checkpoint-711/trainer_state.json (deflated 85%)
  adding: content/experiments/checkpoint-711/rng_state.pth (deflated 28%)
  adding: content/experiments/checkpoint-711/tokenizer_

In [ ]:
# from google.colab import files
# files.download("/content/file.zip")

- Inference

In [41]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating celebrities. In particular, you are
a world-leading expert at answering questions like Jerry from the hit
sitcom Seinfield.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Jerry would say. I have
included some relevant samples of what he actually said in the show.
<</INST>>
""".strip()


def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
""".strip()

In [ ]:
# def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
#     return f"""### Instruction: {system_prompt}

# ### Input:
# {question.strip()}

# ### Response:
# """.strip()

In [42]:
def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


In [ ]:
# test_question = 'Who is your favorite author?'

# test_prompt = generate_testing_prompt(test_question)


In [ ]:
# generate_response(trainer.model,'Who is your favorite author?')

- Inference with the Fine-tuned Model

In [43]:
finetuned_model = PeftModel.from_pretrained(model, OUTPUT_DIR)

In [44]:
test_question = 'Who is your favorite author?'

test_prompt = generate_testing_prompt(test_question)

In [45]:
type(test_prompt)

str

In [46]:
fine_tuned_response = generate_response(finetuned_model,test_prompt)
type(fine_tuned_response)

str

In [47]:
def make_llm(text):
  test_prompt = generate_testing_prompt(text)
  fine_tuned_response = generate_response(finetuned_model,test_prompt)
  return fine_tuned_response


In [48]:
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk


class FinetunedLLM(LLM):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        test_prompt = generate_testing_prompt(prompt)
        fine_tuned_response = generate_response(finetuned_model,test_prompt)
        return fine_tuned_response



    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "finetuned"

- TruLens for evaluation

In [52]:
import locale
import os
import sys

locale.getpreferredencoding = lambda x: "UTF-8"

In [52]:
# locale.setlocale(locale.LC_ALL, "en_US.UTF-8")

'en_US.UTF-8'

In [58]:
# locale.getpreferredencoding??

In [50]:
# locale.getpreferredencoding()

In [51]:
!pip install trulens_eval openai langchain chromadb langchainhub bs4 tiktoken

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.3/751.3 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.3/706.3 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.1 MB/s eta 0:00:00
   ━━

In [53]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [54]:
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are an expert at impersonating celebrities. In particular, you are
a world-leading expert at answering questions like Jerry from the hit
sitcom Seinfield.
<</SYS>>

<<INST>>
Generate a response that sounds as close to what Jerry would say. I have
included some relevant samples of what he actually said in the show.
<</INST>>
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Context:
{custom_format(retriever.invoke(question))}

### Response:
{response}
""".strip()

In [55]:
def create_personality_chain(retriever,llm,system_prompt = DEFAULT_SYSTEM_PROMPT):
  template = f"""### Instruction: {system_prompt}""" + """

### Input:
{question}

### Context:
{context}

### Response:
""".strip()
  prompt = ChatPromptTemplate.from_template(template = template)
  chain = (
    {"context": retriever | custom_format, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

In [56]:
ftuned_llm = FinetunedLLM()

In [57]:
rag_chain= create_personality_chain(retriever,ftuned_llm,system_prompt = DEFAULT_SYSTEM_PROMPT)

In [58]:
rag_chain.invoke("what is the name of your ex girlfriend?")

"\n\n\n\n### Context:\n\n This is Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n Hi, I'm Jerry Seinfeld, the agency sent me.\n\n Hi, I'm Jerry Seinfeld, the agency sent me.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n This is Jerry Seinfeld.\n\n I'm Jerry Seinfeld. Who's this?\n\n\n\n### Response:\n\n\n\n### Context:\n\n This is Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n Hi, I'm Jerry Seinfeld, the agency sent me.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n I'm Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n"

In [59]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets .


In [60]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [61]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "What is the name of your ex girlfriend?")

In [62]:
json_like = tru_record.layout_calls_as_app()

In [63]:
json_like

Munch({'record_id': 'record_hash_5679d3876a8929dc78f3f2d316febe5a', 'app_id': 'Chain1_ChatApplication', 'cost': {'n_requests': 2, 'n_successful_requests': 2, 'n_classes': 0, 'n_tokens': 193, 'n_stream_chunks': 0, 'n_prompt_tokens': 193, 'n_completion_tokens': 0, 'cost': 0.0}, 'perf': {'start_time': '2024-05-12T22:16:09.015015', 'end_time': '2024-05-12T22:16:35.899254'}, 'ts': '2024-05-12T22:16:35.899409', 'tags': '-', 'meta': None, 'main_input': 'What is the name of your ex girlfriend?', 'main_output': "\n\n\n\n### Context:\n\n This is Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n I'm Jerry Seinfeld.\n\n Hello, this is Jerry Seinfeld. Is Elaine there?\n\n I'm Jerry Seinfeld. Who's this?\n\n Hi, I'm Jerry Seinfeld, the agency sent me.\n\n Uh, Winona, it's Jerry Seinfeld.\n\n Uh, Winona, it's Jerry Seinfeld.\n\n\n\n### Response:\n\n\n\n### Context:\n\n This is Jerry Seinfeld.\n\n I'm Jerry Seinfeld.\n\n Hello, this is Jerry Seinfeld. Is E

In [64]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))

    calls_node = Node('Calls')
    tree.add_node(calls_node)

    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)

        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)

    return tree

# Usage
tree = display_call_stack(json_like)
tree

Tree(nodes=(Node(name='Record ID: record_hash_5679d3876a8929dc78f3f2d316febe5a'), Node(name='App ID: Chain1_Ch…

In [65]:
tree

Tree(nodes=(Node(name='Record ID: record_hash_5679d3876a8929dc78f3f2d316febe5a'), Node(name='App ID: Chain1_Ch…

In [97]:
# with tru_recorder as recording:
#     llm_response = rag_chain.invoke("What is the name of your ex girlfriend?")

# display(llm_response)

'\n\n### Context:\n Oh, you want to know about my ex-girlfriend? (chuckles) Well, let me tell you a story.\n\n (chuckles) Oh, boy. Okay, so there was this one... (pauses)\n\n (smirks) Oh, you want to know about my ex-girlfriend? (chuckles) Well, let me tell you a story.\n\n (chuckles) Oh, boy. Okay, so there was this one... (pauses)\n\n (smirks) Oh, you want to know about my ex-girlfriend? (chuckles) Well, let me tell you a story.\n\n (chuckles) Oh, boy. Okay, so there was this one... (pauses)\n\n (smirks) Oh, you want to know about my ex-girlfriend? (chuckles) Well, let me tell you a story.\n\n (chuckles) Oh, boy. Okay, so there was this one... (pauses)\n\n (smirks) Oh, you want to know about my ex-girlfriend? (chuck'

Groundedness per statement in source:   0%|          | 0/14 [00:00<?, ?it/s]

In [87]:
# testList[0]['question']

'Yes, it was purple, I liked it, I dont actually recall considering the buttons.'

In [66]:
# sample = rag_chain.invoke(testList[0]['question'])
# print(sample)

In [67]:
rag_chain.invoke(test_question)

"\n\nI don't know. I'm very caring. I care for people. I care for people so much that I'm afraid they're going to catch fire. You know? Like a volcano. You know? It's all gooey inside. It's like to hold me, you gotta be like a fireman. You gotta be dressed like a fireman. You gotta have the helmet, the pants, the jacket, the boots. You know, like a real fireman. You know what I mean? You know what I mean. I'm just making this up as I go along here. I don't know what I'm talking about. I'm making all this up. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian. I'm a comedian."

In [68]:
counter = 0
with tru_recorder as recording:
    for x in testList:
        test_question = x['question']
        rag_chain.invoke(test_question)
        counter +=1
        if counter > 50:
            break


recs = recording.records
display(recs)

Groundedness per statement in source:   0%|          | 0/29 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/27 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/25 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/32 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/22 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/15 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/25 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/21 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/22 [00:00<?, ?it/s]

[Record(record_id='record_hash_046e2154fe8c21f5f624b3f7621e83d1', app_id='Chain1_ChatApplication', cost=Cost(n_requests=2, n_successful_requests=2, n_classes=0, n_tokens=169, n_stream_chunks=0, n_prompt_tokens=169, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 5, 12, 22, 17, 45, 63095), end_time=datetime.datetime(2024, 5, 12, 22, 18, 11, 578585)), ts=datetime.datetime(2024, 5, 12, 22, 18, 11, 578708), tags='-', meta=None, main_input='(smiling) So, you know, what, what happened?', main_output="\n\nWell, I'm not really that much of a meat eater.\n\n Well, I'm not really that much of a meat eater.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I just feel he should call me.\n\n I don't know. I ju

In [70]:
# # The record of the app invocation can be retrieved from the `recording`:

# rec = recording.get() # use .get if only one record
# # recs = recording.records # use .records if multiple

# display(rec)

In [71]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

# for feedback, feedback_result in rec.wait_for_feedback_results().items():
#     print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

In [72]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,relevance_calls,context_relevance_with_cot_reasons_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_5679d3876a8929dc78f3f2d316febe5a,"""What is the name of your ex girlfriend?""","""\n\n\n\n### Context:\n\n This is Jerry Seinfe...",-,"{""record_id"": ""record_hash_5679d3876a8929dc78f...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:16:09.015015"", ""...",2024-05-12T22:16:35.899409,0.0,0.9,0.000000,[{'args': {'prompt': 'What is the name of your...,[{'args': {'question': 'What is the name of yo...,"[{'args': {'source': [[{'page_content': "": 676...",26,193,0.0
1,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_046e2154fe8c21f5f624b3f7621e83d1,"""(smiling) So, you know, what, what happened?""","""\n\nWell, I'm not really that much of a meat ...",-,"{""record_id"": ""record_hash_046e2154fe8c21f5f62...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:17:45.063095"", ""...",2024-05-12T22:18:11.578708,0.0,0.8,0.206897,"[{'args': {'prompt': '(smiling) So, you know, ...","[{'args': {'question': '(smiling) So, you know...",[{'args': {'source': [[{'page_content': ': 363...,26,169,0.0
2,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_0b27ebb266f17f205cdad6ae81a5023f,"""Oh, nothing happened, but it was...""","""\n\n Oh, it was nothing.\n\n Oh, it was nothi...",-,"{""record_id"": ""record_hash_0b27ebb266f17f205cd...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:18:11.999265"", ""...",2024-05-12T22:18:39.199284,0.0,0.8,0.259259,"[{'args': {'prompt': 'Oh, nothing happened, bu...","[{'args': {'question': 'Oh, nothing happened, ...",[{'args': {'source': [[{'page_content': ': 24\...,27,182,0.0
3,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_4db0a12038e4360917c040341dd9f912,"""This is great!""","""\n\n\n\n### Context:\n This is Jerry Seinfeld...",-,"{""record_id"": ""record_hash_4db0a12038e4360917c...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:18:39.621187"", ""...",2024-05-12T22:19:06.680015,1.0,0.8,0.000000,"[{'args': {'prompt': 'This is great!', 'respon...","[{'args': {'question': 'This is great!', 'cont...",[{'args': {'source': [[{'page_content': ': 111...,27,152,0.0
4,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_709a03ac0c6b1119c11d22e1f2bb7566,"""(whistles disapprovingly) Ho ho ho, Had to? H...","""\n\n (to the phone) Hello? (disappointed) Oh,...",-,"{""record_id"": ""record_hash_709a03ac0c6b1119c11...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:19:07.048695"", ""...",2024-05-12T22:19:34.016035,0.2,0.4,0.000000,[{'args': {'prompt': '(whistles disapprovingly...,[{'args': {'question': '(whistles disapproving...,"[{'args': {'source': [[{'page_content': "": 532...",26,198,0.0


In [73]:
records

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,relevance_calls,context_relevance_with_cot_reasons_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_5679d3876a8929dc78f3f2d316febe5a,"""What is the name of your ex girlfriend?""","""\n\n\n\n### Context:\n\n This is Jerry Seinfe...",-,"{""record_id"": ""record_hash_5679d3876a8929dc78f...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:16:09.015015"", ""...",2024-05-12T22:16:35.899409,0.0,0.9,0.000000,[{'args': {'prompt': 'What is the name of your...,[{'args': {'question': 'What is the name of yo...,"[{'args': {'source': [[{'page_content': "": 676...",26,193,0.0
1,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_046e2154fe8c21f5f624b3f7621e83d1,"""(smiling) So, you know, what, what happened?""","""\n\nWell, I'm not really that much of a meat ...",-,"{""record_id"": ""record_hash_046e2154fe8c21f5f62...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:17:45.063095"", ""...",2024-05-12T22:18:11.578708,0.0,0.8,0.206897,"[{'args': {'prompt': '(smiling) So, you know, ...","[{'args': {'question': '(smiling) So, you know...",[{'args': {'source': [[{'page_content': ': 363...,26,169,0.0
2,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_0b27ebb266f17f205cdad6ae81a5023f,"""Oh, nothing happened, but it was...""","""\n\n Oh, it was nothing.\n\n Oh, it was nothi...",-,"{""record_id"": ""record_hash_0b27ebb266f17f205cd...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:18:11.999265"", ""...",2024-05-12T22:18:39.199284,0.0,0.8,0.259259,"[{'args': {'prompt': 'Oh, nothing happened, bu...","[{'args': {'question': 'Oh, nothing happened, ...",[{'args': {'source': [[{'page_content': ': 24\...,27,182,0.0
3,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_4db0a12038e4360917c040341dd9f912,"""This is great!""","""\n\n\n\n### Context:\n This is Jerry Seinfeld...",-,"{""record_id"": ""record_hash_4db0a12038e4360917c...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:18:39.621187"", ""...",2024-05-12T22:19:06.680015,1.0,0.8,0.000000,"[{'args': {'prompt': 'This is great!', 'respon...","[{'args': {'question': 'This is great!', 'cont...",[{'args': {'source': [[{'page_content': ': 111...,27,152,0.0
4,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_709a03ac0c6b1119c11d22e1f2bb7566,"""(whistles disapprovingly) Ho ho ho, Had to? H...","""\n\n (to the phone) Hello? (disappointed) Oh,...",-,"{""record_id"": ""record_hash_709a03ac0c6b1119c11...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:19:07.048695"", ""...",2024-05-12T22:19:34.016035,0.2,0.4,0.000000,[{'args': {'prompt': '(whistles disapprovingly...,[{'args': {'question': '(whistles disapproving...,"[{'args': {'source': [[{'page_content': "": 532...",26,198,0.0
5,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_e77a6ab068bdd332b8e848209cfd4eb8,"""Youre a back-up, youre a second-line, a just-...","""\n\n\n\n### Context:\n I'm Seinfeld!\n\n This...",-,"{""record_id"": ""record_hash_e77a6ab068bdd332b8e...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-12T22:19:34.444884"

In [74]:
tru.get_leaderboard(app_ids=["Chain1_ChatApplication"])

,groundedness_measure_with_cot_reasons,relevance,context_relevance_with_cot_reasons,latency,total_cost
app_id,,,,,
Chain1_ChatApplication,0.094737,0.215385,0.7,26.923077,0.0


In [75]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
npx: installed 22 in 4.814s

Go to this url and submit the ip given here. your url is: https://quick-shirts-march.loca.lt

  Submit this IP Address: 35.197.143.199



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

- Build the classifier

In [76]:
df = pd.read_csv('scripts.csv')

In [77]:
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

In [78]:
df['Character'].value_counts()

Character
JERRY                  14786
GEORGE                  9708
ELAINE                  7983
KRAMER                  6664
NEWMAN                   640
                       ...  
TAXI DRIVER                1
JERRY (exasperated)        1
PHONE TAPE                 1
MR. SANGER                 1
PROSTITUTE                 1
Name: count, Length: 1639, dtype: int64

In [79]:
#drop na
df = df.dropna()

In [80]:
df.head()

,Character,Dialogue,EpisodeNo,SEID,Season
0,JERRY,Do you know what this is all about? Do you kno...,1.0,S01E01,1.0
1,JERRY,"(pointing at Georges shirt) See, to me, that b...",1.0,S01E01,1.0
2,GEORGE,Are you through?,1.0,S01E01,1.0
3,JERRY,"You do of course try on, when you buy?",1.0,S01E01,1.0
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",1.0,S01E01,1.0


In [81]:
df['target'] = df['Character'].apply(lambda x : 1 if x == 'JERRY' else 0)

In [82]:
X = df['Dialogue']
y = df['target']

In [83]:
# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [84]:
X_train

47739                                   Hey, go tell hell!
29885                                   So she's gorgeous.
36394    I guess I have to bring his car back up to the...
43146    To a woman, sex is like the garbage man. You j...
53803    All right, we can move faster without father o...
                               ...                        
44739                             They were all like that!
54354                           the victim of the robbery]
38164    (As he is signing) Oh, thank you so much. (He ...
860                                             Who is it?
15797    Oh, I'm just gonna go say hi to Cheryl. (walki...
Name: Dialogue, Length: 43684, dtype: object

In [85]:
# tf-idf sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer() # convert text to numbers
X_train_processed = vectorizer.fit_transform(X_train)
X_test_processed = vectorizer.transform(X_test)

# train clf
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_processed, y_train)

# evaluate clf
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test_processed)
accuracy_score(y_test, y_pred)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.729328816042487

In [86]:
def classify(texts):
    if type(texts) == str:
        texts = [texts]
    text_processed = vectorizer.transform(texts)
    num = clf.predict(text_processed)[0]
    return "JERRY" if num == 1 else "Not JERRY"

In [ ]:
# classify(fine_tuned_response)

In [91]:
classifications = []

subset_size = 200

for x in testList[:subset_size]:
    test_question = x['question']
    test_prompt = generate_testing_prompt(test_question)
    fine_tuned_response = generate_response(finetuned_model, test_prompt)
    classification = classify(fine_tuned_response)
    classifications.append(classification)
    print(classification)

jerry_count = np.mean([x == "JERRY" for x in classifications])
print(jerry_count)


Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
Not JERRY
JERRY
Not JERRY
Not JERRY
Not JERR

In [89]:
# classifications = []
# for x in testList:
#   test_question = x['question']
#   test_prompt = generate_testing_prompt(test_question)
#   fine_tuned_response = generate_response(finetuned_model,test_prompt)
#   classification = classify(fine_tuned_response)
#   classifications.append(classification)
#   print(classification)

# print(np.mean([x == "JERRY" for x in classifications]))